In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

print(tf.__version__)

2.4.0


In [11]:
# 와인 데이터셋 불러오기
red = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
white = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep=';')
print(red.head(5)) # quality: 타깃, 앞의 11열: 입력이지만 이진 분류를 위해 quality가 입력값이 됨 
print(white.head(5))

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5 

In [3]:
# 와인 데이터셋 합치기
red['type']=0 # 'type'열 만들고 모두 0으로 만들기 (target_value)
white['type']=1
print(red.head(2))
print(white.head(2))

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70          0.0             1.9      0.076   
1            7.8              0.88          0.0             2.6      0.098   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   

   alcohol  quality  type  
0      9.4        5     0  
1      9.8        5     0  
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.0              0.27         0.36            20.7      0.045   
1            6.3              0.30         0.34             1.6      0.049   

   free sulfur dioxide  total sulfur dioxide  density   pH  sulphates  \
0                 45.0                 170.0    1.001  3.0       0.45   
1                 14.0                 132.0    0.994  3.3       0.49   

   

In [4]:
wine=pd.concat([red,white]) # red wine 데이터와 white wine데이터를 합침 
print(wine.describe()) # 데이터에 대한 정보 출력

       fixed acidity  volatile acidity  citric acid  residual sugar  \
count    6497.000000       6497.000000  6497.000000     6497.000000   
mean        7.215307          0.339666     0.318633        5.443235   
std         1.296434          0.164636     0.145318        4.757804   
min         3.800000          0.080000     0.000000        0.600000   
25%         6.400000          0.230000     0.250000        1.800000   
50%         7.000000          0.290000     0.310000        3.000000   
75%         7.700000          0.400000     0.390000        8.100000   
max        15.900000          1.580000     1.660000       65.800000   

         chlorides  free sulfur dioxide  total sulfur dioxide      density  \
count  6497.000000          6497.000000           6497.000000  6497.000000   
mean      0.056034            30.525319            115.744574     0.994697   
std       0.035034            17.749400             56.521855     0.002999   
min       0.009000             1.000000         

In [5]:
# 데이터 요약 정보
print(wine.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6497 entries, 0 to 4897
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         6497 non-null   float64
 1   volatile acidity      6497 non-null   float64
 2   citric acid           6497 non-null   float64
 3   residual sugar        6497 non-null   float64
 4   chlorides             6497 non-null   float64
 5   free sulfur dioxide   6497 non-null   float64
 6   total sulfur dioxide  6497 non-null   float64
 7   density               6497 non-null   float64
 8   pH                    6497 non-null   float64
 9   sulphates             6497 non-null   float64
 10  alcohol               6497 non-null   float64
 11  quality               6497 non-null   int64  
 12  type                  6497 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 710.6 KB
None


In [6]:
# 데이터 정규화 (값이 균일해야 됨, 너무 스케일이 제각각이면 학습이 잘 안 됨) 
wine_norm=(wine-wine.min())/(wine.max()-wine.min()) # 0~1사이로 정규화가 됨 
print(wine_norm.head())
print(wine_norm.describe())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0       0.297521          0.413333     0.000000        0.019939   0.111296   
1       0.330579          0.533333     0.000000        0.030675   0.147841   
2       0.330579          0.453333     0.024096        0.026074   0.137874   
3       0.611570          0.133333     0.337349        0.019939   0.109635   
4       0.297521          0.413333     0.000000        0.019939   0.111296   

   free sulfur dioxide  total sulfur dioxide   density        pH  sulphates  \
0             0.034722              0.064516  0.206092  0.612403   0.191011   
1             0.083333              0.140553  0.186813  0.372093   0.258427   
2             0.048611              0.110599  0.190669  0.418605   0.241573   
3             0.055556              0.124424  0.209948  0.341085   0.202247   
4             0.034722              0.064516  0.206092  0.612403   0.191011   

    alcohol   quality  type  
0  0.202899  0.333333   0.

In [7]:
# 데이터 섞은 후 numpy array로 변환
wine_shuffle =wine_norm.sample(frac=1) # wine_norm의 데이터를 섞어라 (셔플)
print(wine_shuffle.head())

wine_np=wine_shuffle.to_numpy()
print(wine_np[:5])

      fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
1579       0.198347          0.320000     0.054217        0.016871   0.073090   
3424       0.214876          0.160000     0.162651        0.065951   0.041528   
186        0.297521          0.206667     0.289157        0.021472   0.121262   
446        0.165289          0.133333     0.162651        0.030675   0.074751   
271        0.115702          0.346667     0.042169        0.098160   0.058140   

      free sulfur dioxide  total sulfur dioxide   density        pH  \
1579             0.079861              0.059908  0.133218  0.635659   
3424             0.059028              0.267281  0.086563  0.496124   
186              0.045139              0.140553  0.194525  0.480620   
446              0.100694              0.345622  0.082707  0.627907   
271              0.111111              0.324885  0.140544  0.472868   

      sulphates   alcohol   quality  type  
1579   0.213483  0.478261  0.333333   0.0 

In [9]:
# train 데이터와 test 데이터로 분리
train_idx=int(len(wine_np)*0.8) # 데이터의 개수 * 0.8 = 80% 데이터 
print(train_idx)
# 100개 튜플 -> 입력:[80행, 마지막 열을 뺀 앞까지], 타깃:[80행, 마지막 열]
train_X,train_Y=wine_np[:train_idx,:-1],wine_np[:train_idx,-1]
test_X,test_Y=wine_np[train_idx:,:-1],wine_np[train_idx:,-1]
print(train_X[0])
print(train_Y[0])
print(test_X[0])
print(test_Y[0])
# 원핫인코딩
train_Y=tf.keras.utils.to_categorical(train_Y,num_classes=2)
test_Y=tf.keras.utils.to_categorical(test_Y,num_classes=2)
print(train_Y[0])
print(test_Y[0])

5197
[0.19834711 0.32       0.05421687 0.01687117 0.0730897  0.07986111
 0.05990783 0.13321766 0.63565891 0.21348315 0.47826087 0.33333333]
0.0
[0.23966942 0.1        0.23493976 0.0291411  0.27076412 0.21527778
 0.35023041 0.12704839 0.30232558 0.07865169 0.20289855 0.5       ]
1.0
[1. 0.]
[0. 1.]


In [10]:
model=tf.keras.Sequential([
    tf.keras.layers.Dense(units=48, activation='relu', input_shape=(12,)),
    tf.keras.layers.Dense(units=24,activation='relu'),
    tf.keras.layers.Dense(units=12,activation='relu'),
    tf.keras.layers.Dense(units=2,activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.07),loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# 와인 데이터셋 분류 모델 학습
history=model.fit(train_X,train_Y,epochs=25,batch_size=32,validation_split=0.25)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 48)                624       
_________________________________________________________________
dense_1 (Dense)              (None, 24)                1176      
_________________________________________________________________
dense_2 (Dense)              (None, 12)                300       
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 26        
Total params: 2,126
Trainable params: 2,126
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25
122/122 [==============================] - 2s 11ms/step - loss: 0.2405 - accuracy: 0.8690 - val_loss: 0.1156 - val_accuracy: 0.9669
Epoch 2/25
122/122 [==============================] - 0s 3ms/step - loss: 0.0837 - accuracy: 0.9780 - val_los